In [35]:
!pip install folium
import folium
import pandas as pd
import datetime
from datetime import timedelta


In [ ]:
df_23 = pd.read_csv('2023-04_orca_boardings.csv')

In [ ]:
df_23 = df_23.sort_values(by=['business_date','card_id'], ascending=[True,True])

In [ ]:
df_23

In [ ]:
df_23['device_lat'] = df_23['device_lat'] / 1000000
df_23['device_lng'] = df_23['device_lng'] / 1000000
df_23

In [ ]:
df_23['device_dtm_pacific'] = pd.to_datetime(df_23['device_dtm_pacific'])
df_23['business_date'] = pd.to_datetime(df_23['business_date'])
df_23['time_of_day'] = df_23['device_dtm_pacific'].dt.time


In [ ]:
df_23=df_23[['card_id','business_date','time_of_day','device_lat','device_lng','stop_code','route_number']]

In [ ]:
df_23

In [ ]:
df_23=df_23.dropna(subset=['stop_code'])

In [ ]:
df_23.shape

In [ ]:
import random
def random_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 200), random.randint(0, 200), random.randint(0, 200))


df_23 = df_23.dropna(subset=['device_lat','device_lng'])
num_rows = df_23.shape[0]
date = df_23.iloc[0]['business_date'].date()
card=1
lat=df_23.iloc[0]['device_lat']
lng=df_23.iloc[0]['device_lng']
m = folium.Map(location=[lat, lng], zoom_start=12)
# for i in range(204900,num_rows):
for i in range(num_rows):
    lat=df_23.iloc[i]['device_lat']
    lng=df_23.iloc[i]['device_lng']
    if(df_23.iloc[i]['business_date'].date()==date):
        if(df_23.iloc[i]['card_id']==card) and (i>0):
            folium.Marker([lat, lng], tooltip=f"Stop ID: {df_23.iloc[i]['stop_code']}").add_to(m)
            folium.Marker([df_23.iloc[i-1]['device_lat'], df_23.iloc[i-1]['device_lng']], tooltip=f"Stop ID: {df_23.iloc[i-1]['stop_code']}").add_to(m)
            line = folium.PolyLine(
                locations=[[lat, lng], [df_23.iloc[i-1]['device_lat'], df_23.iloc[i-1]['device_lng']]],
                color=co,
                weight=2,
                tooltip=f'Card ID: {card}'
            )
            line.add_to(m)
            m.save(f'map{date}.html')
            print(m)
        else:
            card=df_23.iloc[i]['card_id']
            co=random_color()
    else:
        date = date + timedelta(days=1)
        m = folium.Map(location=[lat, lng], zoom_start=12)



In [ ]:
trip_count = df_23.groupby('card_id')['business_date'].value_counts().reset_index(name='trip_count')

In [ ]:
trip_count

In [ ]:
count_of_ones = (trip_count['trip_count'] == 1).sum()
print(count_of_ones)
count_of_non_ones = (trip_count['trip_count'] != 1).sum()
count_of_ones/(count_of_ones+count_of_non_ones)*100

In [ ]:
df_23.shape

In [ ]:
df_23=df_23.sort_values(by=['card_id','business_date','time_of_day'], ascending=[True,True,True])

In [ ]:
nan_exists = df_23[['card_id', 'business_date', 'time_of_day']].isna().any()
print(nan_exists)

In [ ]:
print(df_23.shape)
all_numeric = pd.to_numeric(df_23['stop_code'], errors='coerce').notna().all()

if all_numeric:
    print("All values in 'stop_code' are numbers.")
else:
    print("Not all values in 'stop_code' are numbers.")

df_23 = df_23[pd.to_numeric(df_23['stop_code'], errors='coerce').notna()]
print(df_23.shape)

In [36]:
import warnings
warnings.filterwarnings("ignore")
import json
import numpy as np

In [ ]:
df_23

In [ ]:
df_23_timeshift=df_23
am4=pd.to_datetime('04:00:00', format='%H:%M:%S').time()

def shift_time(row):
    if row['time_of_day'] < am4:
        row['business_date_2'] = row['business_date']-pd.DateOffset(days=1)
    else:
        row['business_date_2'] = row['business_date']
    return row

# Apply the time shift
df_23_timeshift = df_23_timeshift.apply(lambda row: shift_time(row), axis=1)

print(df_23_timeshift)

In [ ]:
df_23_timeshift[df_23_timeshift['card_id']==62]

In [ ]:
df_23[df_23['card_id']==62]

In [ ]:
taz={}
am4=pd.to_datetime('04:00:00', format='%H:%M:%S').time()
am0=pd.to_datetime('00:00:00', format='%H:%M:%S').time()

def custom_investigation(group):

    n=len(group['time_of_day'])
    origin=group['stop_code'].iloc[0]
    o = origin
    A=datetime.datetime.combine(group['business_date'].iloc[0].date(),group['time_of_day'].iloc[0])
    for i in range(n-1):
        taz.setdefault(o, [])
        time_diff = datetime.datetime.combine(group['business_date'].iloc[i+1].date(), group['time_of_day'].iloc[i+1]) - A
        if time_diff.total_seconds() > 3600 or i+1==n-1:
            destination=group['stop_code'].iloc[i+1]
            taz.setdefault(destination, [])
            taz[o].append(destination)
            o=destination
            A=datetime.datetime.combine(group['business_date'].iloc[i].date(),group['time_of_day'].iloc[i])
            if (i+1==n-1): taz[destination].append(origin)

    return taz

result = df_23_timeshift.groupby(['card_id', 'business_date_2']).apply(custom_investigation)


with open('output.json', "w") as json_file:
     json.dump(taz, json_file, indent=4)

In [ ]:
df_23_timeshift

In [ ]:
with open('output.json', 'r') as json_file:
    taz = json.load(json_file)

In [ ]:
'35318' in taz['990005']

In [ ]:
len(taz)

In [ ]:
print(df_23['stop_code'].nunique())

In [ ]:
a=0
for key, values in taz.items():
    for value in values:
        a+=1
print(a)

In [ ]:
from collections import Counter


In [ ]:
for key in taz.keys():
    count_dict=dict(Counter(taz[key]))
    tuplee = tuple((key2, count) for key2, count in count_dict.items())
    taz[key]=tuplee

In [ ]:
taz['3034'][0:10]

In [ ]:
with open('output_with_counts.json', "w") as json_file:
     json.dump(taz, json_file, indent=4)

In [ ]:
tazdf = pd.read_csv('filtered_data_with_taz.csv')

In [ ]:
tazdf=tazdf[['card_id','business_date','device_dtm_pacific','device_lat','device_lng','stop_code','TAZ','COUNTY_NM']]
tazdf

In [ ]:
print(tazdf['stop_code'].nunique())
n_stop=tazdf['stop_code'].nunique()

In [ ]:
print(tazdf['TAZ'].nunique())
n_TAZ=tazdf['TAZ'].nunique()

In [ ]:
2557**2

In [ ]:
print(tazdf['TAZ'].isna().sum())


In [ ]:
print(tazdf['stop_code'].isna().sum())


In [ ]:
tazdf['device_dtm_pacific'] = pd.to_datetime(tazdf['device_dtm_pacific'])
tazdf['business_date'] = pd.to_datetime(tazdf['business_date'])
tazdf['time_of_day'] = tazdf['device_dtm_pacific'].dt.time

In [ ]:
tazdf.drop(['device_dtm_pacific'],axis=1)

In [ ]:
tazdf=tazdf.dropna(subset=['stop_code'])
tazdf.shape

In [ ]:
print(tazdf.shape)
all_numeric = pd.to_numeric(tazdf['stop_code'], errors='coerce').notna().all()

if all_numeric:
    print("All values in 'stop_code' are numbers.")
else:
    print("Not all values in 'stop_code' are numbers.")

tazdf = tazdf[pd.to_numeric(tazdf['stop_code'], errors='coerce').notna()]
print(tazdf.shape)

In [ ]:
tazdf['stop_code'].nunique()

Merging stop codes with TAZs

In [ ]:
df = pd.DataFrame({key: pd.Series(value) for key, value in taz.items()})
df = df.fillna(value=pd.NA)
df.to_csv('stop_codes.csv')

In [ ]:
df['3034'].count()

In [ ]:
len(taz['3034'])

In [ ]:
type(df['3034'][1])

In [ ]:
type(tazdf['stop_code'][0])

In [ ]:
stop_code_to_taz_mapping = tazdf.set_index('stop_code')['TAZ'].to_dict()
for column in df.columns:
    if column in stop_code_to_taz_mapping:
        df[column] = df[column].map(stop_code_to_taz_mapping)
        # df=df.rename(columns={column:stop_code_to_taz_mapping[column]})

In [ ]:
df

In [ ]:
taz2={}
stop_code_to_taz_mapping = tazdf.set_index('stop_code')['TAZ'].to_dict()
for key, values in taz.items():
    if key in stop_code_to_taz_mapping:
        for value in values:
            taz2.setdefault(stop_code_to_taz_mapping[key],[])
            taz2[stop_code_to_taz_mapping[key]].append(stop_code_to_taz_mapping[value])

In [ ]:
stop_code_to_taz_mapping['3034']

In [ ]:
taz2[574]

In [ ]:
stop_code_to_taz_mapping['2680']

In [ ]:
with open('final_TAZs.json', "w") as json_file:
     json.dump(taz2, json_file, indent=4)

In [ ]:
for key in taz2.keys():
    count_dict=dict(Counter(taz2[key]))
    tuplee = tuple((key2, count) for key2, count in count_dict.items())
    taz2[key]=tuplee

In [ ]:
with open('final_TAZs_count.json', "w") as json_file:
     json.dump(taz2, json_file, indent=4)

In [ ]:
taz2[574]

In [ ]:
taz2[574][0][1]

In [37]:
with open('output_with_counts.json', 'r') as json_file:
    stops_count = json.load(json_file)

In [38]:
type(stops_count['3034'])

list

In [39]:
len(stops_count)

8921

In [40]:
with open('final_TAZs_count.json', 'r') as json_file:
    TAZ_counts = json.load(json_file)

In [41]:
len(TAZ_counts)

2343

In [42]:
type(TAZ_counts['966.0'])

list

In [43]:
len(TAZ_counts)

2343

In [44]:
TAZ_counts['966.0']

[[1354.0, 115],
 [1350.0, 51],
 [969.0, 3],
 [963.0, 3],
 [628.0, 1],
 [1395.0, 1],
 [1349.0, 5],
 [952.0, 2],
 [967.0, 4],
 [961.0, 1],
 [1396.0, 1],
 [1022.0, 1],
 [973.0, 4],
 [182.0, 1],
 [1398.0, 1],
 [962.0, 7],
 [1351.0, 7],
 [1399.0, 1],
 [1256.0, 1],
 [1345.0, 21],
 [2564.0, 1],
 [657.0, 2],
 [612.0, 2],
 [611.0, 2],
 [1403.0, 1],
 [960.0, 4],
 [971.0, 1],
 [982.0, 2],
 [845.0, 1],
 [512.0, 1],
 [1346.0, 1],
 [1002.0, 1],
 [774.0, 2],
 [1353.0, 6],
 [812.0, 1],
 [537.0, 1],
 [1402.0, 1],
 [983.0, 1],
 [670.0, 2],
 [659.0, 1],
 [665.0, 2],
 [914.0, 1],
 [503.0, 6],
 [3009.0, 1],
 [2596.0, 1],
 [630.0, 1],
 [2634.0, 2],
 [502.0, 2],
 [1411.0, 1],
 [1551.0, 8],
 [254.0, 2],
 [529.0, 1]]

In [45]:
float(list(TAZ_counts.keys())[0])

574.0

In [46]:
TAZ_flows=[]
TAZ_pairs=[]

for key in list(TAZ_counts.keys()):
    for i in range(len(TAZ_counts[key])):
        TAZ_flows.append(TAZ_counts[key][i][1])
        TAZ_pairs.append((float(key),TAZ_counts[key][i][0]))


In [47]:
len(TAZ_flows)

241918

In [48]:
len(TAZ_pairs)

241918

In [49]:
TAZ_pairs.index((966.0,1354.0))

195456

In [50]:
TAZ_flows[TAZ_pairs.index((966.0,1354.0))]

115

In [58]:
TAZ_flows[TAZ_pairs.index((1354.0,966.0))]

55

In [57]:
TAZ_data=pd.DataFrame({'OD Pairs':TAZ_pairs,'Flows':TAZ_flows})
TAZ_data

,OD Pairs,Flows
0,"(574.0, 398.0)",24
1,"(574.0, 512.0)",196
2,"(574.0, 650.0)",5
3,"(574.0, 731.0)",2
4,"(574.0, 521.0)",200
...,...,...
241913,"(2778.0, 395.0)",2
241914,"(2115.0, 3108.0)",1
241915,"(1335.0, 514.0)",2
241916,"(3346.0, 3053.0)",3


In [56]:
list(TAZ_data['OD Pairs'])[0][0]

574.0

In [60]:
TAZ_data.to_csv('TAZ.csv', index=False)

In [61]:
sum(TAZ_flows)

3739588

In [65]:
TAZ_flows.index(max(TAZ_flows))

9768

In [ ]:
total_flow=sum(TAZ_flows)
total_flow

In [ ]:
zero_TAZs=n_TAZ*n_TAZ-len(TAZ_flows)
zero_TAZs

In [ ]:
zero_TAZs/(n_TAZ**2)*100

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(TAZ_flows,bins=50, color='blue', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of TAZ Flows')
# plt.xlim(6000, 8000)
#plt.ylim(0, 1000)

In [ ]:
plt.hist(TAZ_flows,bins=5000, color='blue', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of TAZ flows (trimed at 100)')
plt.axvline(x=200, color='red', linestyle='-', label='y=100')
#plt.axhline(y=20, color='red', linestyle='--', label='y=20')
#plt.axhline(y=10, color='yellow', linestyle='--', label='y=10')
plt.xlim(0, 6000)
plt.ylim(0, 100)

In [ ]:
plt.hist(TAZ_flows,bins=10000, color='blue', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of TAZ flows (trimed at 100)')
#plt.axhline(y=20, color='red', linestyle='--', label='y=20')
#plt.axhline(y=10, color='yellow', linestyle='--', label='y=10')
plt.xlim(0, 200)
#plt.ylim(0, 200)

In [ ]:
plt.hist(TAZ_flows,bins=10000, color='blue', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of TAZ flows (trimed at 100)')
plt.axvline(x=10, color='red', linestyle='-', label='y=100')
#plt.axhline(y=20, color='red', linestyle='--', label='y=20')
#plt.axhline(y=10, color='yellow', linestyle='--', label='y=10')
plt.xlim(0, 50)
# plt.ylim(0, 20000)

Threshold 10:

In [ ]:
threshold_TAZ = [value for value in TAZ_flows if value <= 10]

In [ ]:
len(threshold_TAZ)

In [ ]:
len(TAZ_flows)

Percentage of TAZs having flows less than 10:

In [ ]:
(len(threshold_TAZ)+zero_TAZs)/(len(TAZ_flows)+zero_TAZs)*100


In [ ]:
len(TAZ_flows)+zero_TAZs

In [ ]:
n_TAZ**2

Percentage of TAZ flows being less than 10 compared to total flow:

In [ ]:
sum(threshold_TAZ)/total_flow*100

STOP level:

In [ ]:
stop_flows=[]

for key in stops_count.keys():
    for i in range(len(stops_count[key])):
        stop_flows.append(stops_count[key][i][1])


In [ ]:
len(stop_flows)

In [ ]:
total_flow_stops=sum(stop_flows)
total_flow_stops

In [ ]:
plt.hist(stop_flows,bins=50, color='green', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of STOP Flows')
# plt.xlim(6000, 8000)
#plt.ylim(0, 1000)

In [ ]:
plt.hist(stop_flows,bins=500, color='green', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of STOP Flows (trimed at 1000)')
# plt.axhline(y=100, color='red', linestyle='--', label='y=100')
plt.axvline(x=20, color='red', linestyle='-', label='y=100')
plt.xlim(0, 1000)
plt.ylim(0, 100000)

In [ ]:
plt.hist(stop_flows,bins=10000, color='green', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of STOP Flows (trimed at 1000)')
# plt.axhline(y=100, color='red', linestyle='--', label='y=100')
plt.axvline(x=5, color='red', linestyle='-', label='y=100')
plt.xlim(0, 50)
# plt.ylim(0, 500000)

Threshold 5:

In [ ]:
threshold_stop = [value for value in stop_flows if value <= 5]

Percentage of STOPs having flows less than 5:

In [ ]:
len(threshold_stop)/len(stop_flows)*100

Percentage of STOP flows being less than 5 compared to total flow:

In [ ]:
sum(threshold_stop)/total_flow_stops*100